In [1]:
import torch
import torch_sparse
import numpy as np
from rgnn_at_scale.aggregation import (_sparse_top_k, soft_weighted_medoid, soft_weighted_medoid_k_neighborhood,
                                       weighted_dimwise_median, weighted_medoid, weighted_medoid_k_neighborhood)
from rgnn_at_scale.data import prep_graph

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = 0 if torch.cuda.is_available() else 'cpu'
temperature = 1e-3
device

0

In [4]:
A_sparse_tensor = torch_sparse.SparseTensor.from_dense(torch.load("../pprgo_pytorch/adj_bug.tensor")).to("cpu")
#x = torch_sparse.SparseTensor.from_dense(torch.load("datasets/test_attr_matrix.pt"))
x = torch.load("../pprgo_pytorch/x_bug.tensor").to("cpu")
A_sparse_tensor, x, x.shape

/nfs/homedirs/schmidtt/miniconda3/lib/python3.8/site-packages/torch_sparse/tensor.py:46: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629395347/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  index = mat.nonzero()


(SparseTensor(row=tensor([  0,   0,   0,  ..., 139, 139, 139]),
              col=tensor([   0,    1,    2,  ..., 2024, 2025, 2026]),
              val=tensor([5.0000e-01, 1.7678e-01, 8.8388e+04,  ..., 7.2921e-02, 5.0592e-01,
                            1.3832e-01]),
              size=(140, 2027), nnz=15921, density=5.61%),
 tensor([[-0.0005,  0.0013, -0.0029,  ..., -0.0037, -0.0021, -0.0064],
         [-0.0058,  0.0059, -0.0012,  ..., -0.0004, -0.0018, -0.0010],
         [-0.0084,  0.0092, -0.0050,  ..., -0.0020, -0.0026, -0.0030],
         ...,
         [-0.0074,  0.0075,  0.0026,  ..., -0.0034, -0.0032, -0.0045],
         [-0.0050, -0.0061, -0.0028,  ..., -0.0048,  0.0006, -0.0060],
         [-0.0039, -0.0003, -0.0027,  ...,  0.0024, -0.0027, -0.0110]],
        grad_fn=<CopyBackwards>),
 torch.Size([2027, 7]))

In [6]:
torch.save(x, "tests/data_/x_bug.tensor")

In [5]:
# number of nodes with no outgoing edges 
A_sparse_tensor.size(0) - A_sparse_tensor.sum(-1).nonzero().size(0)

0

In [8]:
medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
            x,
            k=32,
            temperature=temperature,
            threshold_for_dense_if_cpu=0)

top_k_weights
tensor([[8.8388e+04, 8.8388e+04, 8.8388e+04,  ..., 1.7678e-01, 1.7678e-01,
         1.7678e-01],
        [2.3788e+04, 2.3788e+04, 2.3788e+04,  ..., 5.3853e-02, 5.3853e-02,
         5.3853e-02],
        [1.1180e+05, 1.1180e+05, 1.1180e+05,  ..., 7.6037e+04, 5.6600e+03,
         5.6600e+03],
        ...,
        [1.2544e+03, 1.2544e+03, 1.2544e+03,  ..., 5.5243e-03, 5.5243e-03,
         5.5243e-03],
        [1.7678e+05, 1.7678e+05, 3.1416e+04,  ..., 1.0797e-02, 1.0797e-02,
         1.0797e-02],
        [2.6593e+04, 2.6593e+04, 2.6593e+04,  ..., 1.6239e+03, 1.6239e+03,
         1.6239e+03]])
top_k_idx
tensor([[ 368,    4,    2,  ...,  836,  529,  490],
        [1723, 1697, 1762,  ...,   42,   10, 1725],
        [ 661, 1427,   58,  ..., 1673,  927, 1399],
        ...,
        [ 955, 1979, 1810,  ..., 1940,  177, 1952],
        [ 717,  570,  314,  ..., 1967, 1339, 1913],
        [ 732, 1660,  222,  ...,  286, 1201,  631]])


IndexError: index 2029 is out of bounds for dimension 0 with size 2027

In [ ]:
medoids

In [ ]:
A = torch.tensor([[0.5, 0.3, 0, 0.4],
                  [0.3, 0.2, 0, 0],
                  [0, 0, 0.9, 0.3],
                  [0.4, 0, 0.4, 0.4]],
                 #requires_grad=True,
                 dtype=torch.float32).to(device)
x = torch.tensor([[-10, 10, 10],
                  [-1, 1, 1],
                  [0, 0, 0],
                  [10, -10, -10]],
                 #requires_grad=True,
                 dtype=torch.float32).to(device)

A_sparse_tensor = torch_sparse.SparseTensor.from_dense(A).to(device)


In [ ]:
medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
                                                      x,
                                                      k=2,
                                                      temperature=temperature,
                                                      threshold_for_dense_if_cpu=0)
medoids

In [ ]:
graph = prep_graph("cora_ml", device)
attr, adj, labels = graph[:3]
A_sparse_tensor = torch_sparse.SparseTensor.from_torch_sparse_coo_tensor(adj).to(device)
x = attr
A_sparse_tensor, x

In [ ]:
 medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
            x,
            k=2,
            temperature=temperature,
            threshold_for_dense_if_cpu=0)

In [ ]:
medoids

In [ ]:
# this should have the same number of nonzeros
# or at least more of them, but it mustn't have less
medoids.size(0) - medoids.sum(-1).nonzero().size(0)

In [ ]:
# now lets see if it works with a batched adj matrix
A_sparse_tensor = A_sparse_tensor[:512]

In [ ]:
# calc number of nodes with no outgoing edges again
A_sparse_tensor.size(0) - A_sparse_tensor.sum(-1).nonzero().size(0)

In [ ]:
medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
            x,
            k=2,
            temperature=temperature,
            threshold_for_dense_if_cpu=0)

In [ ]:
medoids

In [ ]:
# check num zeros again
medoids.size(0) - medoids.sum(-1).nonzero().size(0)